In [2]:
!pip uninstall -y pandas numpy
!pip install pandas==2.1.4 numpy==1.26.4

Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 94.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
xarray 2025.12.0 requires pandas>=2.2, but you have pandas 2.1.4 w

In [1]:
!pip install ucimlrepo

In [15]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets.values.ravel()

#Utilisation de ravel() comme demandé dans lemessage d'erreur car les modeles attendent une seule dimension

In [34]:
from sklearn.neighbors import KNeighborsClassifier  # Importer KNeighborsClassifier pour la classification par K plus proches voisins
from sklearn.svm import SVC  # Importer SVC pour la classification par Support Vector Machines
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
import pandas as pd

In [31]:
models = [
    KNeighborsClassifier(weights = 'distance'),
    SVC(),
    SGDClassifier(),
    MLPClassifier(max_iter=2000)
]

In [33]:
for model in models :
  score = cross_val_score(model, X, y, cv=5).mean()
  print(f"{model} : {score} ")

KNeighborsClassifier(weights='distance') : 0.9297003570874087 
SVC() : 0.9121720229777983 
SGDClassifier() : 0.8909641360037261 
MLPClassifier(max_iter=2000) : 0.9332091290172334 


In [ ]:
# MLP Classifier est le meilleur

In [17]:
param_grids = [
        {'n_neighbors': [3, 5, 7, 9]},                               # KNeighborsClassifier on test les voisins
        {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10, 100]} ,        #  SVC (noyau et paramètre C)
        {'loss':['hinge', 'log_loss', 'modified_huber'], 'penalty':['l2', 'l1']}  , #SGDClassifier
        {'activation' : ['identity', 'logistic', 'tanh', 'relu'], 'solver' : ['lbfgs', 'sgd', 'adam'], 'hidden_layer_sizes': [(50,), (100,)]} #MLPclassifier
]

In [35]:
list_model_names = []
list_score = []
list_params = []
for model, param_grid in zip(models, param_grids):
    name = model.__class__.__name__
    gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
    print(f"On entraine {name}")
    gs.fit(X, y)
    list_model_names.append(name)
    list_score.append(gs.best_score_)
    list_params.append(gs.best_params_)

df = pd.DataFrame({"score" : list_score, "parametre" : list_params}, index = list_model_names).sort_values('score', ascending = False)

On entraine KNeighborsClassifier
On entraine SVC
On entraine SGDClassifier
On entraine MLPClassifier


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL: .

Increase the number of iterations (max_iter) or sca

In [36]:
df

,score,parametre
SVC,0.952554,"{'C': 100, 'kernel': 'linear'}"
MLPClassifier,0.949045,"{'activation': 'relu', 'hidden_layer_sizes': (..."
KNeighborsClassifier,0.931486,{'n_neighbors': 9}
SGDClassifier,0.915650,"{'loss': 'log_loss', 'penalty': 'l1'}"


In [ ]:
# SVC gagnant  avec comme parametre {'C': 100, 'kernel': 'linear'